In [21]:
from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
import time

Install the qudikernel: python qudikernel.py install

# QuOCS and Qudi with Noise

## **What to do before**

* Update the QuOCS library
* Substitute the logic files in Qudi

To handle the noise in the physical system with QuOCS, we will use the **DCRABAlgorithm** class, now updated with the noisy feature.
In particular, we will use the compensation drift functionality to reset the figure of merit at the beginning of every Super iteration to take into account any new drift term coming from the external noise sources.
Furthermore, a re-evaluation step method is added from the second iteration for every super-iteration. Indeed the measure is repeated n + 1 times, where n is the length of the probabilities list.

The changes to be made are in the algorithm settings and in the evaluation loop (See below). Both changes are marked in bold.

### Creation of the optimization dictionary

The optimization dictionary contains all the settings compulsory for the optimization algorithm in order to run a proper optimization. 

In [22]:
optimization_dictionary = {}

In [23]:
optimization_client_name = "test_dCRAB_Noisy_2_control_fields"

**CHANGE** :Now we will use the **dCRAB Noisy algorithm** which ensures **drift compensation** and handles **fluctuations** 

Number of iteration and super-iterations

In [24]:
# Total number of dCRAB superiteration
super_iteration_number = 1
# Maximum number of iteration per super-iteration
maximum_function_evaluations_number = 30

To activate only the drift compensation add 

In [25]:
optimization_dictionary['algorithm_settings'] = {"algorithm_name": "dCRAB",
                                                 "super_iteration_number": super_iteration_number,
                                                 "max_eval_total": super_iteration_number * maximum_function_evaluations_number,
                                                 "FoM_goal": 0.00001,
                                                 "total_time_lim": 60*1,
                                                 "compensate_drift": {
                                                     "compensate_after_SI": True,
                                                     "compensate_after_minutes": 15
                                                 },
                                                 "random_number_generator":{
                                                     "seed_number":420
                                                 }
}

In [26]:
optimization_dictionary['algorithm_settings']["dsm_settings"] = {'general_settings': {
                                                                    "dsm_algorithm_name": "NelderMead", 
                                                                    'is_adaptive': False
                                                                }, 
                                                                'stopping_criteria': {
                                                                    "max_eval": 100,
                                                                    "time_lim": 20,
                                                                    "xatol": 1e-10, 
                                                                    "frtol": 1e-10}
                                                                }

### Times

We can define here how many times we need to use in the optimization

In [27]:
time_p = {'time_name': 'time_p', 'initial_value': 3.0}

In [28]:
optimization_dictionary["times"] = [time_p]

### Parameters

In [29]:
optimization_dictionary['parameters'] = []

### Pulses

In [30]:
pulse_amplitude = {'pulse_name': 'Amplitude', 
                   'upper_limit': 15.0, 
                   'lower_limit': 0.1, 
                   'bins_number': 101, 
                   'time_name': 'time_p', 
                   'amplitude_variation': 0.5, 
                   'basis': {'basis_name': 'Fourier', 
                             'basis_class': 'Fourier', 
                             'basis_module': 'quocslib.pulses.basis.Fourier', 
                             'basis_vector_number': 5, 
                             'random_super_parameter_distribution': 
                             {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                   'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                   'lower_limit': 0.1, 'upper_limit': 3.0}
                            }, 
                   'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                   'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: np.pi/3 + 0.0*t'}
                  }

In [31]:
pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': 3.14, 
               'lower_limit': -3.14, 
               'bins_number': 101, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.5, 
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': 5, 
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                          'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                          'lower_limit': 0.1, 'upper_limit': 3.0}}, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }

In [32]:
optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

### Put all together and get ready to start the optimization with Qudi-QuOCS

In [33]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}

Load the optimization algorithm into the optimization logic and display it into the GUI

In [34]:
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

#### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

Print the optimization dictionary also here

In [35]:
optimalcontrol.opti_comm_dict

{'optimization_dictionary': {'algorithm_settings': {'algorithm_name': 'dCRAB', 'super_iteration_number': 1, 'max_eval_total': 30, 'FoM_goal': 1e-05, 'total_time_lim': 60, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}, 'dsm_settings': {'general_settings': {'dsm_algorithm_name': 'NelderMead', 'is_adaptive': False}, 'stopping_criteria': {'max_eval': 100, 'time_lim': 20, 'xatol': 1e-10, 'frtol': 1e-10}}}, 'times': [{'time_name': 'time_p', 'initial_value': 3.0}], 'parameters': [], 'pulses': [{'pulse_name': 'Amplitude', 'upper_limit': 15.0, 'lower_limit': 0.1, 'bins_number': 101, 'time_name': 'time_p', 'amplitude_variation': 0.5, 'basis': {'basis_name': 'Fourier', 'basis_class': 'Fourier', 'basis_module': 'quocslib.pulses.basis.Fourier', 'basis_vector_number': 5, 'random_super_parameter_distribution': {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 'distribution_module': 'quocslib.pulses.su

**CHANGE** : We have to provide the standard deviation to the dCRAB algorithm. 

To do so we use the function:

``` fomlogic.update_fom(fom, std, status_code=0)```

after the measurement. **std** stays for the experimental standard deviation.

Loop over 4 identical optimizations

In [36]:
# Number of different optimizations
opt_number = 4

In [38]:
for ii in range(opt_number):
    print("Optimization number {0}".format(ii))
    controlslogic.are_pulses_calculated = False
    ######################################################################################################
    # Parameters and Settings
    ######################################################################################################

    # This section is devoted to the initialization in the pulsed logic and optimization logic of the main
    # settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
    # Iteration, controls and figure of merit to compare with QuOCS
    # Just an example for debug
    args_dict = {"is_noisy": False}
    qubit = OneQubit2Fields(args_dict)

    ######################################################################################################
    # Measurement
    ######################################################################################################
    optimalcontrol.start_optimization()
    print("Starting the optimization in 10 secs")
    time.sleep(10)
    # Just a time to check for latent time
    last_time_fom = time.time()
    # repeat the whole process until its manually stopped or QuOCS finished the optimization
    # Wait few seconds before starting to get and return data
    while not optimizationlogic.handle_exit_obj.is_user_running:
        time.sleep(0.1)
        if (time.time() - last_time_fom) > 30:
            print("Problem at the beginning! Surpassed the 30 secs")
            break

    while  optimizationlogic.handle_exit_obj.is_user_running == True:
        # wait until QuOCS optimizes the controls
        # print("Wait until the controls logic gives the controls")
        while not controlslogic.are_pulses_calculated:
            time.sleep(0.1)
            # If the waiting time exceed 10 seconds left stop the optimization
            if time.time() - last_time_fom > 20:
                print("Too much time... Exit!")
                optimizationlogic.handle_exit_obj.is_user_running = False
                break

        # If the optimization is not running just exit from the loop
        if optimizationlogic.handle_exit_obj.is_user_running == False:
            print('Stopping the measurement!')
            # Update the pulse with the maximum number
            # COMMENT THE FOLLOWING LINE
            # fomlogic.update_fom(10**10, status_code=-1)
            break
        #######################################################################################################
        # Get the Controls
        #######################################################################################################
        # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
        controlslogic.are_pulses_calculated = False
        # Get the controls from the controls logic
        pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
        #######################################################################################################
        # Perform the measurement
        #######################################################################################################
        #
        #
        #######################################################################################################
        # Calculate the figure of merit and the standard deviation
        #######################################################################################################
        # calculate the Figure of Merit
        fom_dict = qubit.get_FoM(pulses, parameters, timegrids)
        # Extract the fom and std
        fom, std = fom_dict["FoM"], fom_dict["std"]
        #######################################################################################################
        # Return the figure of merit
        #######################################################################################################
        # Update the figure of merit and the standard deviation to the fom logic
        fomlogic.update_fom(fom, std, status_code=0)
        # update the last time the fom is calculated
        last_time_fom = time.time()
        #######################################################################################################
        # Optional part
        #######################################################################################################
        # Print the data just for debug purpose
        print("FoM: {fom}, Std: {std}, status_code: {status_code}".format(fom=fom, std=std, status_code=0))
    print("Optimization nr {0} finished".format(ii))

Optimization number 0
Starting the optimization in 10 secs
FoM: 0.2037600845653127, Std: 0.0, status_code: 0
FoM: 0.3756856031143432, Std: 0.0, status_code: 0
FoM: 0.023635853791326378, Std: 0.0, status_code: 0
FoM: 0.3922109902219141, Std: 0.0, status_code: 0
FoM: 0.47190702645457183, Std: 0.0, status_code: 0
FoM: 2.2762416423960374e-05, Std: 0.0, status_code: 0
FoM: 0.4741860854670238, Std: 0.0, status_code: 0
FoM: 0.2886943900334934, Std: 0.0, status_code: 0
FoM: 0.27229287867073937, Std: 0.0, status_code: 0
FoM: 0.06015179900647538, Std: 0.0, status_code: 0
FoM: 0.04175471761755645, Std: 0.0, status_code: 0
FoM: 0.046841656670789655, Std: 0.0, status_code: 0
FoM: 0.39174983511046546, Std: 0.0, status_code: 0
FoM: 0.21882567657224716, Std: 0.0, status_code: 0
FoM: 0.24564347762521144, Std: 0.0, status_code: 0
FoM: 0.4609346310494774, Std: 0.0, status_code: 0
FoM: 0.5021662061961452, Std: 0.0, status_code: 0
FoM: 0.5705734252451189, Std: 0.0, status_code: 0
FoM: 0.5496460870550004, S